# 0. Setup: El Dataset "Sucio"
Antes de empezar, cargamos un DataFrame que simula la realidad: datos con símbolos de moneda, fechas mal escritas, nulos y texto desperdiciando memoria.

In [1]:
import pandas as pd
import numpy as np

# Datos simulados de una tienda (sucios y desordenados)
data = {
    'id_ticket': ['1001', '1002', '1003', '1004', '1005'],
    'fecha': ['15/01/2023', '16-Jan-2023', '2023/02/01', '18/01/2023', '20/01/2023'],
    'producto': ['Laptop Gamer', 'Mouse', 'Laptop Gamer', 'Teclado', 'Mouse'],
    'precio': ['$1500.00', '25.50', '1,500.00', '$40.00', '25.50'], # Texto con símbolos
    'descuento': ['Si', 'No', 'No', 'Si', 'No'], # Texto binario
    'stock': [5, 10, np.nan, 50, 8] # Enteros con un nulo (NaN)
}

# Creamos el DataFrame
df_data = pd.DataFrame(data)
print("--- Estado Inicial (Todo es Object/Texto) ---")
print(df_data.info())

--- Estado Inicial (Todo es Object/Texto) ---
<class 'pandas.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_ticket  5 non-null      str    
 1   fecha      5 non-null      str    
 2   producto   5 non-null      str    
 3   precio     5 non-null      str    
 4   descuento  5 non-null      str    
 5   stock      4 non-null      float64
dtypes: float64(1), str(5)
memory usage: 526.0 bytes
None


# Ejemplo 1: El "Cirujano" (Limpieza y Conversión)

Teoría: Entender que .astype() falla si el dato tiene "basura" (como signos $). Debemos usar herramientas quirúrgicas (pd.to_numeric) y conversores inteligentes de tiempo (pd.to_datetime).

In [2]:
print("------- Limpieza de Precios --------")
# 1. Quitamos la basura visual ($ y ,)
# Usamos métodos de string (.str)
df_data['precio'] = df_data['precio'].astype(str).str.replace('$', '').str.replace(',', '')

# 2. Convertimos a número (Type Casting)
df_data['precio'] = pd.to_numeric(df_data['precio'])


print("Precios convertidos (Ahora podemos sumar):")
print(df_data['precio'].sum())

print("\n------- Conversión de Fechas -------")
# 3. Convertimos a Datetime (Inteligencia Automática)
# dayfirst=True ayuda a Pandas a entender que 15/01 es 15 de enero, no Mes 15.
df_data['fecha'] = pd.to_datetime(
    df_data['fecha'],
    format='mixed',
    dayfirst=True
)

print(df_data[['precio', 'fecha']].dtypes)

------- Limpieza de Precios --------
Precios convertidos (Ahora podemos sumar):
3091.0

------- Conversión de Fechas -------
precio           float64
fecha     datetime64[us]
dtype: object


### Explicación línea por línea:

**`df_data['precio'].astype(str).str.replace('$','').str.replace(',','')`**
Convierte la columna a texto y elimina el símbolo `$` y las comas. Esto limpia caracteres no numéricos que impedirían la conversión a número.

**`pd.to_numeric(df_data['precio'])`**
Convierte el texto limpio (ej. `"1500.00"`) en un número real (`1500.0`). Permite hacer operaciones matemáticas como suma o promedio.

**`df_data['precio'].sum()`**
Suma todos los valores numéricos de la columna `precio`.

**`pd.to_datetime(df_data['fecha'], format='mixed', dayfirst=True)`**
Convierte la columna `fecha` a tipo `datetime64`. `format='mixed'` permite múltiples formatos (ej. `15/01/2023` y `16-Jan-2023`) y `dayfirst=True` indica que el día va antes del mes.

**`df_data[['precio','fecha']].dtypes`**
Muestra los tipos de datos finales para verificar que `precio` es numérico y `fecha` es datetime.


# Ejemplo 2: Optimización de Memoria (category y bool)

Teoría: Entender que el texto (object) es el enemigo número uno de la memoria RAM. Si una palabra se repite millones de veces, desperdiciamos espacio. Debemos usar category (para crear un diccionario numérico interno) y bool (para reducir datos binarios a su mínima expresión: 0 y 1).

In [3]:
print("-----Estado Inicial(Tipos de dato)-----")
print(df_data[['producto','descuento']].dtypes)

print("\n ----- 1. Optimización de Categorías-----")
# El texto de "Laptop Gamer" se repite. Lo convertimos a Categoría #Pandas guardará internamente un número en lugar de todo el texto
df_data['producto'] = df_data['producto'].astype('category')

print("\n ------2. Optimización de Booleanos ------")
#Paso A: Mapear el texto "Si"/"No" a True/False
mapa_logico = {'Si':True, 'No':False}
df_data['descuento'] = df_data['descuento'].map(mapa_logico)

# Paso B: Forzar el tipo de dato 'bool' para reducir memoría al mínimo
df_data['descuento']=df_data['descuento'].astype('bool')

print("Nuevo tipo de 'descuento':",df_data['descuento'].dtypes)
print("\n ----DataFrame Optimizado----")
print(df_data[['producto','descuento']])

-----Estado Inicial(Tipos de dato)-----
producto     str
descuento    str
dtype: object

 ----- 1. Optimización de Categorías-----

 ------2. Optimización de Booleanos ------
Nuevo tipo de 'descuento': bool

 ----DataFrame Optimizado----
       producto  descuento
0  Laptop Gamer       True
1         Mouse      False
2  Laptop Gamer      False
3       Teclado       True
4         Mouse      False


Explicación línea por línea:

    df_data['producto'].astype('category'): Analiza la columna buscando valores repetidos. Crea un índice interno donde "Laptop" es 0, "Mouse" es 1, etc. Esto reduce el uso de memoria drásticamente (hasta un 90%) en columnas con poca variedad de texto (baja cardinalidad).

    df_data['descuento'].map(...): Funciona como un traductor manual. Busca la palabra "Si" y la reemplaza por True; busca "No" y la reemplaza por False. Es necesario porque Python no sabe automáticamente que "No" significa Falso.

    df_data['descuento'].astype('bool'): Toma esos valores True/False y fuerza el almacenamiento a nivel de bit. Es el tipo de dato más ligero que existe. Si usáramos enteros (0/1) o texto, estaríamos desperdiciando bytes.

# Ejemplo 3: El Nuevo Estándar (Int64)

Teoría: En el pasado, Pandas tenía una limitación técnica grave: los números enteros (int) no podían convivir con valores vacíos (NaN). Si tu columna tenía un solo dato faltante, Pandas convertía automáticamente toda la columna a decimales (float).

    El Problema: Un ID 1005 se convertía en 1005.0. Esto rompía bases de datos que esperaban enteros estrictos.

    La Solución: El tipo Int64 (con "I" mayúscula). Es un "Entero Nullable" que permite guardar números enteros y nulos en la misma columna sin corromper los datos.

In [4]:
print("---- Estado Inicial (Stock-----")
#Observa como pandas convirtió el stock a float (5.0) solo porque hay un NAN
print(df_data['stock'])
print("Tipo actual:", df_data['stock'].dtypes)

print("\n ------ Manejo de Eneteros con Nulos(Int64)----")
#Forma Incorrecta(La vieja Escuela)
# Si intentas esto: df_data['stock'].astype('int'), daría ERROR.
# Porque los 'int' clásicos no saben qué hacer con el NaN.

# FORMA CORRECTA (El Estándar Moderno)
# Usamos 'Int64' (Capitalizado) para permitir nulos.
df_data['stock'] = df_data['stock'].astype('Int64')

print("Nuevo tipo de 'stock':", df_data['stock'].dtypes)
print("\n---- Resultado final (Enteros Limpios)")
print(df_data['stock'])

---- Estado Inicial (Stock-----
0     5.0
1    10.0
2     NaN
3    50.0
4     8.0
Name: stock, dtype: float64
Tipo actual: float64

 ------ Manejo de Eneteros con Nulos(Int64)----
Nuevo tipo de 'stock': Int64

---- Resultado final (Enteros Limpios)
0       5
1      10
2    <NA>
3      50
4       8
Name: stock, dtype: Int64


Explicación línea por línea:

    df_data['stock'] (Inicial): Al imprimirlo al principio, verás que los números tienen punto decimal (5.0, 10.0). Esto es el comportamiento por defecto de NumPy/Pandas antiguo para manejar el espacio vacío (NaN).

    .astype('Int64'):

        La "I" Mayúscula: Es la clave. Diferencia al tipo moderno (Int64) del antiguo (int64).

        Acción: Fuerza a la columna a comportarse como enteros estrictos.

        Resultado: El 5.0 vuelve a ser 5. El NaN se mantiene como <NA>. Esto garantiza que tus IDs o contadores sean precisos y compatibles con otros sistemas.